<a href="https://colab.research.google.com/github/achalgarg14/session_3/blob/main/Assignment_Part2_Session_3_EVA8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
print(device)

cuda


In [43]:
class Network2(nn.Module):
    def __init__(self):
        super(Network2, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0) 
        self.bn1 = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout(0.05)
        
        self.conv2 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.drop2 = nn.Dropout(0.05)
        
        self.conv3 = nn.Conv2d(16, 8, 1, padding=0)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn4 = nn.BatchNorm2d(16)
        self.drop4 = nn.Dropout(0.05)
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=0)
        self.bn5 = nn.BatchNorm2d(32)
        self.drop5 = nn.Dropout(0.05)
        
        self.conv6 = nn.Conv2d(32, 8, 1, padding=0)

        self.conv7 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn7 = nn.BatchNorm2d(16)
        self.drop7 = nn.Dropout(0.05)
        
        self.conv8 = nn.Conv2d(16, 32, 3, padding=0)
        
        self.pool2 = nn.AvgPool2d(4,4)
        
        self.fc1 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.drop1(self.bn1(F.relu(self.conv1(x))))
        # print(x.shape)
        x = self.drop2(self.bn2(F.relu(self.conv2(x))))
        # print(x.shape)
        x = self.conv3(x)
        # print(x.shape)
        x = self.pool1(x)
        # print(x.shape)
        x = self.drop4(self.bn4(F.relu(self.conv4(x))))
        # print(x.shape)
        x = self.drop5(self.bn5(F.relu(self.conv5(x))))
        # print(x.shape)
        x = self.conv6(x)
        # print(x.shape)
        x = self.drop7(self.bn7(F.relu(self.conv7(x))))
        # print('7',x.shape)
        x = self.conv8(x)
        # print(x.shape)
        x = self.pool2(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return F.log_softmax(x,dim=-1)
     

In [44]:
mod = Network2().to(device)
summary(mod, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,168
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7            [-1, 8, 24, 24]             136
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,168
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 32, 8, 8]           4,640
      BatchNorm2d-13             [-1, 32, 8, 8]              64
          Dropout-14             [-1, 3

In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [46]:
model4 = Network2().to(device)
optimizer = optim.SGD(model4.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model4, device, train_loader, optimizer, epoch)
    test(model4, device, test_loader)

epoch = 1 loss=0.16506293416023254 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.90it/s]



Test set: Average loss: 0.0544, Accuracy: 9841/10000 (98.41%)



epoch = 2 loss=0.04259498417377472 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.32it/s]



Test set: Average loss: 0.0508, Accuracy: 9842/10000 (98.42%)



epoch = 3 loss=0.02991672046482563 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.89it/s]


...


Test set: Average loss: 0.0216, Accuracy: 9929/10000 (99.29%)



epoch = 18 loss=0.01293314527720213 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.99it/s]



Test set: Average loss: 0.0189, Accuracy: 9933/10000 (99.33%)



epoch = 19 loss=0.0003926095087081194 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.39it/s]



Test set: Average loss: 0.0192, Accuracy: 9938/10000 (99.38%)

